#### @ tool 데코레이터로 랭체인에 함수 연결하기

- @tool 데코레이터 사용하여 함수를 도구로 변환할 수 있다.
- 함수를 랭체인에서 외부 도구로 등록, 언어 모델이 함수를 호출하고 사용할 수 있게 해줌.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage(content ="안녕? 잘 지냈니?")])

AIMessage(content='안녕! 잘 지냈어. 너는 어떻게 지내?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 15, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnSJ0fRf9Q2aPpN8yDkCq4pgM0f6T', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--71de6a4c-ad77-4cf7-ae09-8bc479a30b3e-0', usage_metadata={'input_tokens': 15, 'output_tokens': 14, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
# tool 함수 데코ㅔ이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone:str, location:str) -> str:
    """현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야함.
        location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 llm 답변 생성에 사용됨

    """
    tz = pytz.timezone(timezone)
    
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f"{timezone} ({location}) 현재 시각 {now}"
    print(location_and_local_time)
    return location_and_local_time

In [12]:
# 도구를 tools 리스트에 추가하고 tool_dict에 추가
tools = [get_current_time,]
tool_dict = {"get_current_time":get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [10]:
from langchain_core.messages import SystemMessage

# 사용자의 질문과 도구를 사용해 언어 모델 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 tools만 사용할 수 있다."),
    HumanMessage("부산 시간은 지금 몇시야?")
]

#llm_with_tools 사용하여 질문에 언어 모델 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# 생성된 언어 모델 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 tools만 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산 시간은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FJXvp8fFcdFZdQ8PPMKJxHxd', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 124, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnSQh06xJXmJPxXABmk4MFbjJqG6O', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--407be21a-00e7-498d-96a2-2b664360421c-0', tool_calls=[{'

In [13]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재 시각 2025-06-29 01:11:17


[SystemMessage(content='너는 사용자의 질문에 tools만 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산 시간은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FJXvp8fFcdFZdQ8PPMKJxHxd', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 124, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnSQh06xJXmJPxXABmk4MFbjJqG6O', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--407be21a-00e7-498d-96a2-2b664360421c-0', tool_calls=[

In [15]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2025년 6월 29일 01시 11분 17초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 180, 'total_tokens': 209, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnSRzZzhyAutv7PvwC4yWna3cpBnq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--56754f94-42f1-4247-ab11-e0534fdd69d2-0', usage_metadata={'input_tokens': 180, 'output_tokens': 29, 'total_tokens': 209, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})